# Paris Districts
## Analysis of businesses and environments
This notebook is part of the capstone project of the IBM Data Science certification

Olivier LUBET - January 2019

## Business problem

~ Fictional context ~

Vaugirard, the 15th district of Paris has always been the most populated one, due to its large ground and the proximity with the famous station “Gare de Montparnasse”. It used to be a very attractive place for companies, especially most of “Insurances” companies where located there. 

But during lasts years, larges companies preferred to leave this district for more attracting ones like the old center (2nd district of “Bourse”) or for the West (17th).

The mayor of the 15th district missioned our company to identify the main reason of this move, to study districts in Paris, comparing the companies and their environments.

The main goal of this report is to find insights that can be directly translated into action at the level of political decisions.

## Organization of this notebook
1. Package imports
1. Acquiring data
    1. Web Scraping WIKIPEDIA
    1. Geocoding Paris Distrincts
    1. Venues data with Foursquare
    1. Companies data with SIRENE
    1. Public equipment with Open Data Paris
1. Feature engineering and clustering
    1. Clustering districts by companies size
    1. Clustering districts by venues
    1. Clustering district by public equipment

# Packages import

In [126]:
#!conda install -c conda-forge beautifulsoup4 --yes 
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge googlemaps --yes 

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [73]:
import urllib.request as url
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import re
import types
from ibm_botocore.client import Config
import ibm_boto3


# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

#https://github.com/googlemaps/google-maps-services-python
#import googlemaps

# map rendering library
import folium 

import seaborn as sns
import matplotlib.pyplot as plt

# Web Scraping WIKIPEDIA

In [74]:
w_url = 'https://fr.wikipedia.org/wiki/Arrondissements_de_Paris'
w_page = requests.get(w_url).text
soup = bs(w_page,'lxml')

In [75]:
w_class='wikitable sortable alternance'
rows = []
for (index,w_row) in enumerate(soup.find('table',{'class':w_class}).find('tbody').find_all('tr')):
    row = [x.text.replace('\n','') for x in w_row.find_all('td')]
    if (len(row)==15 or len(row)==16) and index<23:
        #print(len(row),row)
        rows.append([
            75000+int(re.findall(r'\d+', row[0])[0]),
            row[1].replace('[note 1]','').split(' ')[0].split(',')[0],
            int(row[9].replace('\xa0',''))])

df_postcode = pd.DataFrame(rows,columns = ['District','Name','Population'])
print(df_postcode.shape)
df_postcode.head(20)

(20, 3)


,District,Name,Population
0,75001,Louvre,16252
1,75002,Bourse,20260
2,75003,Temple,34788
3,75004,Hôtel-de-Ville,27487
4,75005,Panthéon,59108
5,75006,Luxembourg,40916
6,75007,Palais-Bourbon,52512
7,75008,Élysée,36453
8,75009,Opéra,59629
9,75010,Entrepôt,91932


# Geocoding Paris Distrincts

In [76]:
geolocator = Nominatim()

locations=[]
for index,row in df_postcode.iterrows():
    address = '{}, France'.format(row['District'])
    print(address)
    location = geolocator.geocode(address)
    
    locations.append([
        row['District'],
        location.latitude,
        location.longitude
    ])

locations

75001, France
75002, France
75003, France
75004, France
75005, France
75006, France
75007, France
75008, France
75009, France
75010, France
75011, France
75012, France
75013, France
75014, France
75015, France
75016, France
75017, France
75018, France
75019, France
75020, France


[[75001, 48.86188205, 2.34789584670077],
 [75002, 48.8674618029886, 2.34421013128989],
 [75003, 48.8625799623988, 2.36030765173359],
 [75004, 48.8553611, 2.35838566687213],
 [75005, 48.85061825, 2.34394624567144],
 [75006, 48.8515782237626, 2.33645336655214],
 [75007, 48.85595765, 2.31238441710881],
 [75008, 48.8760477, 2.3156587],
 [75009, 48.88137175, 2.33357002448946],
 [75010, 48.8818594, 2.3540793],
 [75011, 48.8616667, 2.38080146851893],
 [75012, 48.8426346902994, 2.38751923856023],
 [75013, 48.8293171520109, 2.36251454406887],
 [75014, 48.8285901, 2.3075414],
 [75015, 48.8423974, 2.277443],
 [75016, 48.8564994, 2.2748522],
 [75017, 48.8848133, 2.3028393],
 [75018, 48.8878574, 2.3534543],
 [75019, 48.8768285, 2.3941052],
 [75020, 48.8628144675025, 2.40068834777642]]

In [77]:
df_geo=pd.DataFrame(data=locations,columns=['District','lat','lng'])


In [78]:
df_paris = df_postcode.set_index('District').join(df_geo.set_index('District')).reset_index()
df_paris

,District,Name,Population,lat,lng
0,75001,Louvre,16252,48.861882,2.347896
1,75002,Bourse,20260,48.867462,2.344210
2,75003,Temple,34788,48.862580,2.360308
3,75004,Hôtel-de-Ville,27487,48.855361,2.358386
4,75005,Panthéon,59108,48.850618,2.343946
5,75006,Luxembourg,40916,48.851578,2.336453
6,75007,Palais-Bourbon,52512,48.855958,2.312384
7,75008,Élysée,36453,48.876048,2.315659
8,75009,Opéra,59629,48.881372,2.333570
9,75010,Entrepôt,91932,48.881859,2.354079


## Mapping Paris

In [79]:
address = '75001, France'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of 75001, France are 48.86188205, 2.34789584670077.


In [80]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for index,row in df_paris.iterrows():
    district,name,population,lat, lng = row.values
    label = '{} ({}), pop:{}'.format(name, district, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

# Venues data with Foursquare

In [81]:
# The code was removed by Watson Studio for sharing.

In [82]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [83]:
def getNearbyVenues(df):
    
    radius=1000
    venues_list=[]
    for (index,(district,name,population,lat, lng)) in df.iterrows():
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                district, 
                name,
                population,
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except err:
            print("Oops! ",err)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                             'Name',
                             'Population',
                              'Latitude', 
                              'Longitude', 
                              'Venue', 
                              'Venue Latitude', 
                              'Venue Longitude', 
                              'Venue Category']
    
    return(nearby_venues)

In [84]:
paris_venues = getNearbyVenues(df_paris)

Louvre
Bourse
Temple
Hôtel-de-Ville
Panthéon
Luxembourg
Palais-Bourbon
Élysée
Opéra
Entrepôt
Popincourt
Reuilly
Gobelins
Observatoire
Vaugirard
Passy
Batignolles-Monceau
Buttes-Montmartre
Buttes-Chaumont
Ménilmontant


In [85]:
paris_venues.head()

,District,Name,Population,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,75001,Louvre,16252,48.861882,2.347896,Café Belleville,48.861210,2.346806,Café
1,75001,Louvre,16252,48.861882,2.347896,LEGO® Store,48.861818,2.347191,Toy / Game Store
2,75001,Louvre,16252,48.861882,2.347896,Pirouette,48.862876,2.348135,French Restaurant
3,75001,Louvre,16252,48.861882,2.347896,Raviolis Chinois Nord-Est,48.862844,2.349621,Chinese Restaurant
4,75001,Louvre,16252,48.861882,2.347896,Sergeant Paper,48.860665,2.350435,Art Gallery


In [86]:
paris_venues.shape

(1903, 9)

In [87]:
paris_venues.to_csv('paris_venues.csv',';')

In [88]:
!ls -al

total 232
drwxr-x--- 2 dsxuser dsxuser   4096 Jan 15 23:42 .
drwx------ 1 dsxuser dsxuser   4096 Jan 15 23:39 ..
-rw-r----- 1 dsxuser dsxuser 227821 Jan 16 00:14 paris_venues.csv


In [89]:
#paris_venues = pd.read_csv('paris_venues.csv',delimiter =';')
#paris_venues.head()

In [90]:
from IPython.display import HTML
import base64

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)



In [91]:
create_download_link(paris_venues)

## Mapping Venues (only a small subset)

In [92]:
map_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for index,row in paris_venues.head(300).iterrows():
    district,name,population,lat, lng,venue,vlat,vlng,vcat = row.values
    label = '{} ({}) - {}'.format(venue, vcat, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [vlat, vlng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  
    
map_venues

# Companies data with SIRENE

In [93]:
# The code was removed by Watson Studio for sharing.

In [94]:
df_sirene = pd.read_csv(body,sep =';',encoding ='cp1252')
df_sirene.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SIREN,NIC,L1_NORMALISEE,L2_NORMALISEE,L3_NORMALISEE,L4_NORMALISEE,L5_NORMALISEE,L6_NORMALISEE,L7_NORMALISEE,L1_DECLAREE,...,ESAAPEN,ESASEC1N,ESASEC2N,ESASEC3N,ESASEC4N,VMAJ,VMAJ1,VMAJ2,VMAJ3,DATEMAJ
0,36510865,37,SOMDIAA,NaN,NaN,39 RUE JEAN JACQUES ROUSSEAU,BP 2122,75021 PARIS CEDEX 01,FRANCE,SOMDIAA,...,4621Z,4621Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-08T00:00:00
1,38933347,15,SARL NOUVELLES MESSAGERIES PRESSE PARI,NaN,NaN,52 RUE JACQUES HILLAIRET,NaN,75012 PARIS,FRANCE,SARL NOUVELLES MESSAGERIES PRESSE PARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-06-04T00:00:00
2,38960910,16,M LE DIRECTEUR DE LA CANCAVA,NaN,NaN,28 BOULEVARD DE GRENELLE,NaN,75015 PARIS,FRANCE,M. LE DIRECTEUR DE LA CANCAVA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-05T00:00:00
3,39035605,11,MR LE DR DE LA STE ASSURANCES GENERA,S SUR LA VIE HUMAINE,NaN,41 RUE DE CHATEAUDUN,NaN,75009 PARIS,FRANCE,MR LE DR DE LA STE ASSURANCES GENERA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-10T00:00:00
4,39037049,10,SYND COPR DU 8 18 AVENUE MONTAI,E PARIS REP PA,NaN,16 AVENUE MONTAIGNE,NaN,75008 PARIS,FRANCE,SYND COPR DU 8 18 AVENUE MONTAI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-04-09T00:00:00


In [95]:
df_sirene.shape

(11728, 100)

In [96]:
df_sirene_sel=df_sirene[['CODPOS','LIBAPEN','LIBTEFET']]
df_sirene_sel.head()

,CODPOS,LIBAPEN,LIBTEFET
0,75001,"Comm. de gros céréales, tabac non manuf. et al...",20 à 49 salariés
1,75012,Activités combinées de soutien lié aux bâtiments,20 à 49 salariés
2,75015,Administration d'immeubles et autres biens imm...,20 à 49 salariés
3,75009,Activités combinées de soutien lié aux bâtiments,20 à 49 salariés
4,75008,Activités combinées de soutien lié aux bâtiments,20 à 49 salariés


# Public equipment with Open Data Paris

In [97]:
url='https://opendata.paris.fr/explore/dataset/equipements_de_proximite/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true'
df_public = pd.read_csv(url,sep=";").dropna(subset=['AP_cp'])

In [98]:
df_public.head()

,Mairie gestionnaire,Type d'équipement,Désignation_longue,AP_num,AP_cnu,AP_cp,Ap_voie
0,MAIRIE DU 20E,Ecole maternelle & annexes,ECOLE MATERNELLE 31 RUE OLIVIER METRA,31.0,NaN,75020.0,RUE OLIVIER METRA
1,MAIRIE DU 20E,Ecole maternelle & annexes,ECOLE MATERNELLE 32 RUE PALI-KAO,32.0,NaN,75020.0,RUE DE PALI-KAO
2,MAIRIE DU 20E,Ecole maternelle & annexes,ECOLE MATERNELLE 94 RUE COURONNES,94.0,NaN,75020.0,RUE DES COURONNES
3,MAIRIE DU 20E,Ecole élémentaire & annexes,ECOLE ÉLÉMENTAIRE 31 RUE ETIENNE DOLET,31.0,NaN,75020.0,RUE ETIENNE DOLET
4,MAIRIE DU 20E,Ecole élémentaire & annexes,ECOLE ÉLÉMENTAIRE 38 RUE TOURTILLE,38.0,NaN,75020.0,RUE DE TOURTILLE


In [99]:
df_public.shape

(2162, 7)

# Feature engineering and clustering
# Clustering districts by companies size

In [100]:
df_comp = pd.get_dummies(df_sirene_sel[['LIBTEFET']], prefix="", prefix_sep="")
df_comp['District']=df_sirene_sel.CODPOS
df_comp.head()

,1 000 à 1 999 salariés,10 000 salariés et plus,100 à 199 salariés,2 000 à 4 999 salariés,20 à 49 salariés,200 à 249 salariés,250 à 499 salariés,5 000 à 9 999 salariés,50 à 99 salariés,500 à 999 salariés,District
0,0,0,0,0,1,0,0,0,0,0,75001
1,0,0,0,0,1,0,0,0,0,0,75012
2,0,0,0,0,1,0,0,0,0,0,75015
3,0,0,0,0,1,0,0,0,0,0,75009
4,0,0,0,0,1,0,0,0,0,0,75008


In [101]:
df_comp_grp = df_comp.groupby('District').mean().reset_index()
df_comp_grp.head()

,District,1 000 à 1 999 salariés,10 000 salariés et plus,100 à 199 salariés,2 000 à 4 999 salariés,20 à 49 salariés,200 à 249 salariés,250 à 499 salariés,5 000 à 9 999 salariés,50 à 99 salariés,500 à 999 salariés
0,75001,0.003752,0.000000,0.088180,0.003752,0.636023,0.022514,0.022514,0.000000,0.202627,0.020638
1,75002,0.003150,0.000000,0.100787,0.000000,0.658268,0.022047,0.028346,0.000000,0.182677,0.004724
2,75003,0.004630,0.000000,0.055556,0.000000,0.750000,0.009259,0.023148,0.000000,0.152778,0.004630
3,75004,0.018605,0.004651,0.102326,0.004651,0.655814,0.009302,0.027907,0.004651,0.153488,0.018605
4,75005,0.014337,0.000000,0.100358,0.007168,0.616487,0.028674,0.043011,0.000000,0.168459,0.021505


In [102]:
kclusters = 4

df_comp_droped = df_comp_grp.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_comp_droped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 3, 2, 3, 1, 3, 1, 1, 1, 3, 2, 0, 0, 1, 0, 3, 3, 3, 1, 3], dtype=int32)

In [103]:
df_comp_grp['Cluster']=kmeans.labels_
df_comp_grp.sort_values('Cluster')[['District','Cluster']]

,District,Cluster
11,75012,0
12,75013,0
14,75015,0
0,75001,1
4,75005,1
6,75007,1
7,75008,1
8,75009,1
18,75019,1
13,75014,1


- District 15 is in the cluster '0', as 12th and 13th district
- Districts 2 and 16 are colocated in the same 3rd cluster

In [104]:
df_comp_analysis = df_comp_grp.drop('District',1).groupby('Cluster').mean()

df_comp_analysis['L'] = df_comp_grp['10 000 salariés et plus']+df_comp_grp['5 000 à 9 999 salariés']+df_comp_grp['2 000 à 4 999 salariés']+df_comp_grp['1 000 à 1 999 salariés']
df_comp_analysis['M'] = df_comp_grp['200 à 249 salariés']+df_comp_grp['250 à 499 salariés']+df_comp_grp['500 à 999 salariés']
df_comp_analysis['S'] = df_comp_grp['20 à 49 salariés']+df_comp_grp['50 à 99 salariés']+df_comp_grp['100 à 199 salariés']

df_comp_analysis.sort_values('L',ascending =False)[['L','M','S']].reset_index()

,Cluster,L,M,S
0,3,0.032558,0.055814,0.911628
1,0,0.007505,0.065666,0.926829
2,2,0.004630,0.037037,0.958333
3,1,0.003150,0.055118,0.941732


- The analysis of clusters shows that the 3rd one is the most attractive for big companies, confirming the ideas that 2nd and 16th districts are gaining fields over the 15th district.
- The 15th district is not part of this cluster, but should be considered as a good challenger, as the cluster '0' is well represented with Large companies.

# Clustering districts by venues

In [105]:
df_venues = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")
df_venues['District']=paris_venues.District
df_venues.head()

,African Restaurant,Alsatian Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio,District
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75001
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75001
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75001
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75001
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75001


In [106]:
df_venues_grp = df_venues.groupby('District').mean().reset_index()
df_venues_grp.head()

,District,African Restaurant,Alsatian Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,75001,0.0,0.01,0.00,0.00,0.02,0.01,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.03,0.01,0.01,0.00
1,75002,0.0,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.01,0.0,0.0,0.00,0.0,0.06,0.02,0.01,0.00
2,75003,0.0,0.00,0.00,0.01,0.03,0.02,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.02,0.0,0.03,0.00,0.00,0.01
3,75004,0.0,0.00,0.00,0.00,0.03,0.02,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.02,0.00,0.00,0.00
4,75005,0.0,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.0,...,0.0,0.00,0.0,0.0,0.01,0.0,0.04,0.01,0.00,0.00


In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [108]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
paris_venues_sorted = pd.DataFrame(columns=columns)
paris_venues_sorted['District'] = df_venues_grp['District']

for ind in np.arange(df_venues_grp.shape[0]):
    paris_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_venues_grp.iloc[ind, :], num_top_venues)

paris_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75001,French Restaurant,Cocktail Bar,Plaza,Bakery,Restaurant,Coffee Shop,Burger Joint,Italian Restaurant,Wine Bar,Art Gallery
1,75002,French Restaurant,Wine Bar,Boutique,Japanese Restaurant,Cocktail Bar,Hotel,Restaurant,Pedestrian Plaza,Bar,Italian Restaurant
2,75003,French Restaurant,Coffee Shop,Bistro,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Clothing Store,Burger Joint,Sandwich Place,Café
3,75004,French Restaurant,Ice Cream Shop,Garden,Hotel,Pastry Shop,Plaza,Historic Site,Restaurant,Art Gallery,Clothing Store
4,75005,French Restaurant,Plaza,Bookstore,Hotel,Ice Cream Shop,Wine Bar,Burger Joint,Seafood Restaurant,Restaurant,Pub
5,75006,French Restaurant,Hotel,Italian Restaurant,Plaza,Wine Bar,Restaurant,Cocktail Bar,Chocolate Shop,Café,Pastry Shop
6,75007,French Restaurant,Hotel,Plaza,Italian Restaurant,Café,Historic Site,Garden,History Museum,Cocktail Bar,Ice Cream Shop
7,75008,French Restaurant,Hotel,Italian Restaurant,Art Gallery,Bakery,Bar,Salad Place,Thai Restaurant,Coffee Shop,Theater
8,75009,Hotel,French Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Bar,Bistro,Japanese Restaurant,Music Venue,Cheese Shop
9,75010,French Restaurant,Indian Restaurant,Japanese Restaurant,Restaurant,Italian Restaurant,Bakery,Hotel,Pizza Place,Vegetarian / Vegan Restaurant,Bar


In [109]:
kclusters = 4

df_venues_droped = paris_venues_sorted.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_venues_grp)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 3, 3, 3, 3], dtype=int32)

In [110]:
paris_venues_sorted['Cluster'] = kmeans.labels_
print(paris_venues_sorted.shape)
paris_venues_sorted.sort_values('Cluster')

(20, 12)


,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
15,75016,French Restaurant,Italian Restaurant,Hotel,Bakery,Japanese Restaurant,Plaza,Bistro,Garden,Brasserie,Sandwich Place,0
14,75015,French Restaurant,Italian Restaurant,Bakery,Japanese Restaurant,Hotel,Bistro,Persian Restaurant,Korean Restaurant,Park,Restaurant,0
13,75014,French Restaurant,Hotel,Bakery,Bistro,Italian Restaurant,Vietnamese Restaurant,Pub,Pizza Place,Bike Rental / Bike Share,Stadium,0
12,75013,Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,Chinese Restaurant,French Restaurant,Hotel,Japanese Restaurant,Indian Restaurant,Cantonese Restaurant,Cambodian Restaurant,0
11,75012,French Restaurant,Bistro,Chinese Restaurant,Hotel,Plaza,Italian Restaurant,Wine Bar,Bar,Middle Eastern Restaurant,Farmers Market,0
0,75001,French Restaurant,Cocktail Bar,Plaza,Bakery,Restaurant,Coffee Shop,Burger Joint,Italian Restaurant,Wine Bar,Art Gallery,1
1,75002,French Restaurant,Wine Bar,Boutique,Japanese Restaurant,Cocktail Bar,Hotel,Restaurant,Pedestrian Plaza,Bar,Italian Restaurant,1
2,75003,French Restaurant,Coffee Shop,Bistro,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Clothing Store,Burger Joint,Sandwich Place,Café,1
3,75004,French Restaurant,Ice Cream Shop,Garden,Hotel,Pastry Shop,Plaza,Historic Site,Restaurant,Art Gallery,Clothing Store,1
4,75005,French Restaurant,Plaza,Bookstore,Hotel,Ice Cream Shop,Wine Bar,Burger Joint,Seafood Restaurant,Restaurant,Pub,1


### French restaurant
As you can see, French restaurants are for every district in the first or second place of most commons venues. The only exception: the 13th district, known as "china town" of Paris, with a majority of Vietnamese restaurant and Thai ones. In cluster analysis, we will no longer talk about "French restaurants".

### Cluster 3: The Hyper center of Paris
District from 1 to 5 are in this cluster, with a large presence of "tourist oriented" shops (ice cream, wine, cocktail), as well as art gallery or historic sites

- District 2 is in this cluster, showing us that famous places, with history and a trend to art could attract large companies

### Cluster 1: a night in Paris
Districts from 6 to 9 are the ones with the most Hotel, and a large availability of bars.

### Cluster 2: place to eat
These districts are in majority suited with restaurants. We can only notice that in the most commons venues, there is no place for Art, History, boutiques or markets

### Cluster 0: place to live
In these districts we can notice that bakeries, or markets are in the most commons venues. An intuition can be that these districts are the most residentials ones.

- 15th and 17th districts are part of this cluster, so a deeper investigation should be done to enlighten the differences between these districts

## Finding differences between 15th and 17th district

In [111]:
col = df_venues_grp.drop('District',1).columns
df_differences = pd.DataFrame(data=np.zeros((1,len(col))),columns=col)
print(df_differences.shape)

for index,col in enumerate(df_differences.columns):
    #print(index,col,df_venues_grp[df_venues_grp['District']==75017][col].sum())
    df_differences.iloc[0,index] = float(
        df_venues_grp[df_venues_grp['District']==75017][col].sum()
        -df_venues_grp[df_venues_grp['District']==75015][col].sum()
    )

df_differences = df_differences.T.reset_index()
df_differences.columns=['Venue','Diff']

(1, 209)


## Venues present in 17th districts that may not be developed enough in the 15th

In [112]:
df_differences.sort_values('Diff',ascending=False).head(20)

,Venue,Diff
96,Hotel,0.05
75,French Restaurant,0.03
105,Italian Restaurant,0.02
65,Farmers Market,0.02
28,Burger Joint,0.02
168,Seafood Restaurant,0.02
47,Concert Hall,0.02
11,Bar,0.02
190,Tea Room,0.02
59,Diner,0.02


## Venues present in 15th districts that may be too much developed compared to the 17th

In [113]:
df_differences.sort_values('Diff',ascending=True).head(20)

,Venue,Diff
145,Persian Restaurant,-0.04
106,Japanese Restaurant,-0.03
112,Korean Restaurant,-0.03
88,Gym / Fitness Center,-0.02
19,Bistro,-0.02
123,Miscellaneous Shop,-0.02
16,Beer Garden,-0.02
58,Dessert Shop,-0.02
44,Coffee Shop,-0.02
42,Clothing Store,-0.02


# Clustering district by public equipment

In [114]:
df_public.head()

,Mairie gestionnaire,Type d'équipement,Désignation_longue,AP_num,AP_cnu,AP_cp,Ap_voie
0,MAIRIE DU 20E,Ecole maternelle & annexes,ECOLE MATERNELLE 31 RUE OLIVIER METRA,31.0,NaN,75020.0,RUE OLIVIER METRA
1,MAIRIE DU 20E,Ecole maternelle & annexes,ECOLE MATERNELLE 32 RUE PALI-KAO,32.0,NaN,75020.0,RUE DE PALI-KAO
2,MAIRIE DU 20E,Ecole maternelle & annexes,ECOLE MATERNELLE 94 RUE COURONNES,94.0,NaN,75020.0,RUE DES COURONNES
3,MAIRIE DU 20E,Ecole élémentaire & annexes,ECOLE ÉLÉMENTAIRE 31 RUE ETIENNE DOLET,31.0,NaN,75020.0,RUE ETIENNE DOLET
4,MAIRIE DU 20E,Ecole élémentaire & annexes,ECOLE ÉLÉMENTAIRE 38 RUE TOURTILLE,38.0,NaN,75020.0,RUE DE TOURTILLE


In [115]:
df_public_dum = pd.get_dummies(df_public[["Type d'équipement"]], prefix="", prefix_sep="")
df_public_dum['District']=df_public.AP_cp.astype('int64')
df_public_dum_grp = df_public_dum.groupby('District').mean().reset_index()
df_public_dum_grp.head()

,District,Aires de roller,Aires sportives en accès libre,Annexe de Mairie d'arrondissement,Antenne action jeunes information,Bain-douche,Bassin-école,Bibliothèque,Centre d'animation,Conservatoire,...,"Promenade ouverte, mail planté, jardin, square",Salle polyvalente (multi-usages),Salles de sports,Stade,Talus d'espace ferroviaire,Talus planté,Terrain d'éducation physique,Terrain de boules,Terrain de tennis,"Terre-plein, jardinière"
0,75001,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.105263,0.000000,0.000000,...,0.052632,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,75002,0.0,0.0,0.000000,0.0,0.000000,0.034483,0.034483,0.000000,0.000000,...,0.068966,0.000000,0.034483,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,75003,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.025641,0.000000,0.000000,...,0.179487,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,75004,0.0,0.0,0.020408,0.0,0.040816,0.000000,0.040816,0.020408,0.000000,...,0.204082,0.020408,0.000000,0.0,0.0,0.0,0.040816,0.0,0.0,0.0
4,75005,0.0,0.0,0.000000,0.0,0.013333,0.000000,0.013333,0.026667,0.013333,...,0.200000,0.000000,0.013333,0.0,0.0,0.0,0.026667,0.0,0.0,0.0


In [116]:
df_public_dum['District'].unique().shape

(20,)

In [117]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Equipment'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Equipment'.format(ind+1))

# create a new dataframe
df_public_sorted = pd.DataFrame(columns=columns)
df_public_sorted['District'] = df_public_dum_grp['District']

for ind in np.arange(df_venues_grp.shape[0]):
    df_public_sorted.iloc[ind, 1:] = return_most_common_venues(df_public_dum_grp.iloc[ind, :], num_top_venues)

df_public_sorted

,District,1st Most Common Equipment,2nd Most Common Equipment,3rd Most Common Equipment,4th Most Common Equipment,5th Most Common Equipment,6th Most Common Equipment,7th Most Common Equipment,8th Most Common Equipment,9th Most Common Equipment,10th Most Common Equipment
0,75001,"Jardinet décoratif, plate-bande",Crèche collective,Ecole maternelle & annexes,Ecole élémentaire & annexes,Bibliothèque,Halte-garderie,Mairie d'arrondissement,"Maison des associations, locaux associatifs",Ecole polyvalente,Gymnase
1,75002,"Jardinet décoratif, plate-bande",Ecole élémentaire & annexes,Ecole maternelle & annexes,Crèche collective,Halte-garderie,Ecole polyvalente,"Promenade ouverte, mail planté, jardin, square",Mairie d'arrondissement,Gymnase,Bassin-école
2,75003,"Promenade ouverte, mail planté, jardin, square",Ecole élémentaire & annexes,Ecole maternelle & annexes,Crèche collective,Halte-garderie,"Jardinet décoratif, plate-bande","Maison des associations, locaux associatifs",Gymnase,Bibliothèque,Ecole polyvalente
3,75004,"Promenade ouverte, mail planté, jardin, square",Ecole élémentaire & annexes,Crèche collective,Ecole maternelle & annexes,Halte-garderie,Terrain d'éducation physique,Ecole polyvalente,Bain-douche,Bibliothèque,"Maison des associations, locaux associatifs"
4,75005,"Promenade ouverte, mail planté, jardin, square",Crèche collective,Ecole élémentaire & annexes,Ecole maternelle & annexes,"Jardinet décoratif, plate-bande",Gymnase,Halte-garderie,Médiathèque,Centre d'animation,Terrain d'éducation physique
5,75006,"Promenade ouverte, mail planté, jardin, square",Crèche collective,Ecole élémentaire & annexes,"Jardinet décoratif, plate-bande",Ecole maternelle & annexes,Halte-garderie,Gymnase,Centre d'animation,"Maison des associations, locaux associatifs",Médiathèque
6,75007,Ecole maternelle & annexes,"Promenade ouverte, mail planté, jardin, square","Jardinet décoratif, plate-bande",Ecole élémentaire & annexes,Crèche collective,Médiathèque,Gymnase,Salles de sports,Mairie d'arrondissement,Ecole polyvalente
7,75008,"Promenade ouverte, mail planté, jardin, square",Crèche collective,"Jardinet décoratif, plate-bande",Ecole polyvalente,Ecole élémentaire & annexes,Ecole maternelle & annexes,Halte-garderie,Médiathèque,Mairie d'arrondissement,Gymnase
8,75009,Ecole maternelle & annexes,Crèche collective,Ecole élémentaire & annexes,Halte-garderie,"Promenade ouverte, mail planté, jardin, square",Gymnase,Terrain d'éducation physique,"Jardinet décoratif, plate-bande",Salles de sports,Piscine
9,75010,Ecole élémentaire & annexes,Ecole maternelle & annexes,Crèche collective,"Promenade ouverte, mail planté, jardin, square","Jardinet décoratif, plate-bande",Gymnase,Halte-garderie,Terrain d'éducation physique,Centre d'animation,Ecole polyvalente


In [118]:
kclusters = 4

df_public_droped = df_public_dum_grp.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_public_droped)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_
df_public_sorted['Cluster'] = kmeans.labels_
print(df_public_sorted.shape)
df_public_sorted.sort_values('District')[['District','Cluster']]

(20, 12)


,District,Cluster
0,75001,3
1,75002,3
2,75003,1
3,75004,0
4,75005,0
5,75006,1
6,75007,1
7,75008,1
8,75009,2
9,75010,2


## Fail : no real discrimination between these clusters, too much 'children oriented' equipments

In [119]:
df_public_scored = df_public.groupby("Type d'équipement").count()[['Mairie gestionnaire']].reset_index().rename(columns={'Mairie gestionnaire':'Nb'}).sort_values('Nb',ascending=False)
df_public_scored.head(10)

,Type d'équipement,Nb
31,"Promenade ouverte, mail planté, jardin, square",401
11,Ecole maternelle & annexes,318
13,Ecole élémentaire & annexes,305
9,Crèche collective,283
22,"Jardinet décoratif, plate-bande",100
19,Halte-garderie,97
18,Gymnase,86
37,Terrain d'éducation physique,61
7,Centre d'animation,54
12,Ecole polyvalente,51


In [120]:
df_public_filtered = df_public_scored[df_public_scored['Nb']<95].set_index("Type d'équipement").join(df_public.set_index("Type d'équipement")).reset_index()
df_public_filtered.head()

,Type d'équipement,Nb,Mairie gestionnaire,Désignation_longue,AP_num,AP_cnu,AP_cp,Ap_voie
0,Aires de roller,2,MAIRIE DU 12E,ROLLER PARK BERCY,10.0,NaN,75012.0,PLACE LEONARD BERNSTEIN
1,Aires de roller,2,MAIRIE DU 18E,SKATE PARK DE LA CHAPELLE,22.0,NaN,75018.0,BOULEVARD DE LA CHAPELLE
2,Aires sportives en accès libre,2,MAIRIE DU 20E,TEP LUQUET,2.0,NaN,75020.0,RUE DU TRANSVAAL
3,Aires sportives en accès libre,2,MAIRIE DU 19E,AIRE DE JEUX BOULEVARD D'ALGERIE,9.0,NaN,75019.0,BOULEVARD D' ALGERIE
4,Annexe de Mairie d'arrondissement,5,MAIRIE DU 9E,PARKING DE L'HOTEL DROUOT 12-14 RUE CHAUCHAT,12.0,NaN,75009.0,RUE CHAUCHAT


In [121]:
df_public_dum = pd.get_dummies(df_public_filtered[["Type d'équipement"]], prefix="", prefix_sep="")
df_public_dum['District']=df_public_filtered.AP_cp.astype('int64')
df_public_dum_grp = df_public_dum.groupby('District').mean().reset_index()

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Equipment'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Equipment'.format(ind+1))

# create a new dataframe
df_public_sorted = pd.DataFrame(columns=columns)
df_public_sorted['District'] = df_public_dum_grp['District']

for ind in np.arange(df_venues_grp.shape[0]):
    df_public_sorted.iloc[ind, 1:] = return_most_common_venues(df_public_dum_grp.iloc[ind, :], num_top_venues)

    
df_public_sorted['Cluster'] = kmeans.labels_
df_public_sorted.sort_values('Cluster')

,District,1st Most Common Equipment,2nd Most Common Equipment,3rd Most Common Equipment,4th Most Common Equipment,5th Most Common Equipment,6th Most Common Equipment,7th Most Common Equipment,8th Most Common Equipment,9th Most Common Equipment,10th Most Common Equipment,Cluster
19,75020,Ecole polyvalente,Gymnase,Terrain d'éducation physique,Antenne action jeunes information,Jardin d'enfants OPAC,Bibliothèque,"Maison des associations, locaux associatifs",Bain-douche,Crèche familiale,Talus planté,0
3,75004,Terrain d'éducation physique,Bain-douche,Ecole polyvalente,Bibliothèque,Pigeonnier,Mairie d'arrondissement,Gymnase,Annexe de Mairie d'arrondissement,Salle polyvalente (multi-usages),Piscine,0
4,75005,Gymnase,Médiathèque,Terrain d'éducation physique,Centre d'animation,Bain-douche,Salles de sports,Bibliothèque,Conservatoire,Crèche familiale,"Maison des associations, locaux associatifs",0
16,75017,Gymnase,Ecole polyvalente,Terrain de tennis,"Terre-plein, jardinière",Antenne action jeunes information,Conservatoire,Centre d'animation,Piscine,Médiathèque,Stade,0
15,75016,Ecole polyvalente,Crèche familiale,Terrain d'éducation physique,Centre d'animation,Bibliothèque,Gymnase,Conservatoire,Terrain de tennis,Mairie d'arrondissement,"Maison des associations, locaux associatifs",0
14,75015,Gymnase,Terrain d'éducation physique,Centre d'animation,Médiathèque,Crèche familiale,Piscine,Ecole polyvalente,Annexe de Mairie d'arrondissement,Salles de sports,Antenne action jeunes information,0
13,75014,"Terre-plein, jardinière",Gymnase,"Maison des associations, locaux associatifs",Crèche familiale,Centre d'animation,Ecole polyvalente,Jardin d'enfants OPAC,Médiathèque,Piscine,Pigeonnier,0
12,75013,Gymnase,Terrain d'éducation physique,Centre d'animation,Ecole polyvalente,"Terre-plein, jardinière",Crèche familiale,Jardin d'enfants OPAC,Antenne action jeunes information,Stade,Equipements multi-accueil,0
2,75003,"Maison des associations, locaux associatifs",Gymnase,Crèche familiale,Espace jeune,Mairie d'arrondissement,Bibliothèque,Ecole polyvalente,Pigeonnier,"Terre-plein, jardinière",Equipements sportifs d'extérieur,1
5,75006,"Maison des associations, locaux associatifs",Centre d'animation,Gymnase,Conservatoire,Médiathèque,Mairie d'arrondissement,Equipements multi-accueil,Espace jeune,Equipements sportifs d'extérieur,Ecole polyvalente,1



- Cluster 3 : strong presence of swimming pool, musical conservatory, sports area, animation centers, and decoratives gardens -> luxury borough (2nd is included)
- Cluster 2 : lack of decorative gardens, replaced by solid ground and planter -> residentials low levels
- Cluster 1 : strong presence of gymnases and other sports equipments -> 
- Cluster 0 : strong presence of libraries, decoratives gardens and associative structures -> residentials with high levels (15th and 17th are included)

##  Finding the differences between 15th and 17th

In [122]:
df_public_dum = pd.get_dummies(df_public[["Type d'équipement"]], prefix="", prefix_sep="")
df_public_dum['District']=df_public.AP_cp.astype('int64')
df_public_dum_grp = df_public_dum.groupby('District').mean().reset_index()
df_public_dum_grp.head()

,District,Aires de roller,Aires sportives en accès libre,Annexe de Mairie d'arrondissement,Antenne action jeunes information,Bain-douche,Bassin-école,Bibliothèque,Centre d'animation,Conservatoire,...,"Promenade ouverte, mail planté, jardin, square",Salle polyvalente (multi-usages),Salles de sports,Stade,Talus d'espace ferroviaire,Talus planté,Terrain d'éducation physique,Terrain de boules,Terrain de tennis,"Terre-plein, jardinière"
0,75001,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.105263,0.000000,0.000000,...,0.052632,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,75002,0.0,0.0,0.000000,0.0,0.000000,0.034483,0.034483,0.000000,0.000000,...,0.068966,0.000000,0.034483,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,75003,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.025641,0.000000,0.000000,...,0.179487,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,75004,0.0,0.0,0.020408,0.0,0.040816,0.000000,0.040816,0.020408,0.000000,...,0.204082,0.020408,0.000000,0.0,0.0,0.0,0.040816,0.0,0.0,0.0
4,75005,0.0,0.0,0.000000,0.0,0.013333,0.000000,0.013333,0.026667,0.013333,...,0.200000,0.000000,0.013333,0.0,0.0,0.0,0.026667,0.0,0.0,0.0


In [123]:
col = df_public_dum_grp.drop('District',1).columns
df_differences = pd.DataFrame(data=np.zeros((1,len(col))),columns=col)
print(df_differences.shape)

for index,col in enumerate(df_differences.columns):
    #print(index,col,df_venues_grp[df_venues_grp['District']==75017][col].sum())
    df_differences.iloc[0,index] = float(
        df_public_dum_grp[df_public_dum_grp['District']==75017][col].sum()
        -df_public_dum_grp[df_public_dum_grp['District']==75015][col].sum()
    )

df_differences = df_differences.T.reset_index()
df_differences.columns=['Venue','Diff']

(1, 41)


## Equipements available in the 17th that may not be developped enough in the 15th

In [124]:
df_differences.sort_values('Diff',ascending=False).head(10)

,Venue,Diff
19,Halte-garderie,0.040850
39,Terrain de tennis,0.029197
12,Ecole polyvalente,0.024800
40,"Terre-plein, jardinière",0.014599
3,Antenne action jeunes information,0.008751
38,Terrain de boules,0.008751
34,Stade,0.008751
8,Conservatoire,0.008751
23,Jardins d'enfants,0.007299
30,Piscine,0.002903


## Equipements maybe too much  developped in the 15th

In [125]:
df_differences.sort_values('Diff',ascending=True).head(10)

,Venue,Diff
13,Ecole élémentaire & annexes,-0.027959
37,Terrain d'éducation physique,-0.026337
11,Ecole maternelle & annexes,-0.022154
9,Crèche collective,-0.020660
18,Gymnase,-0.016135
7,Centre d'animation,-0.014641
2,Annexe de Mairie d'arrondissement,-0.011696
26,Mur d'escalade d'extérieur,-0.005848
4,Bain-douche,-0.005848
14,Equipements multi-accueil,-0.005848
